In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Datos de conexión
DATABASE_TYPE = 'postgresql'
USER = 'ricardo.torres'
PASSWORD = r'UW`bv&&rg>2!kwo\eOaD' # Usamos raw string para evitar errores de escape
HOST = '34.123.217.240'
PORT = '5432'
DATABASE = 'credito'

# Crear la conexión con SQLAlchemy
engine = create_engine(f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Escribir la consulta
query = "SELECT * FROM CREDITS LIMIT 5;"

# Leer la tabla en un DataFrame
try:
    df = pd.read_sql_query(query, con=engine)
    print(df)  # Mostrar los datos
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")


       id                                  uuid payment_type  \
0  544086  42ff64ef-0558-4755-83a4-d06b14adf02c      monthly   
1  654817  26f27dff-5024-4d7f-9671-a5767c3fffe1      monthly   
2  606265  6f791b12-fc93-41d0-9293-0fd576fa87b1      monthly   
3  616999  eae0b303-0b0d-4de0-b3bb-e7db328fd498      monthly   
4  844495  513f49a9-f6ee-44ae-b641-811bd1c8bcda      monthly   

   annual_interest_rate opening_date  term  customer_id         inserted_at  \
0                 50.04   2023-09-06     8       155008 2023-09-04 19:01:50   
1                 27.00   2021-06-30    70       183442 2024-05-01 01:26:55   
2                 42.26   2019-12-27    14         4239 2024-04-25 21:09:44   
3                 50.18   2022-06-08    12       143359 2024-04-26 00:44:36   
4                 72.00   2025-02-07     6       210010 2025-02-11 19:48:22   

           updated_at paydays  ... closing_date closing_reason    batch  \
0 2025-02-24 03:57:58    [12]  ...   2024-03-14        partial  b